In [2]:
import pandas as pd
import numpy as np
import os

In [72]:
def get_diff_bin(data):
    average_bin, bin_0, bin_24 = [], [], []

    for row in range(24, data.shape[0], 25):
        bin_24.append(data.loc[row, 'bin_volume'])

    for row in range(0, data.shape[0], 25):
        bin_0.append(data.loc[row, 'bin_volume'])
        average_bin.append(data.loc[row+ 1: row+ 24, 'daily_volume'].mean())
    return average_bin, bin_0, bin_24


def transformer(data):
    transformed_nums = [-1 if num < 0 else 1 if num > 0 else 0 for num in data]
    return transformed_nums

average_bin, bin_0, bin_24 = get_diff_bin(data_46)
# print(len(average_bin), len(bin_0), len(bin_24))

diff_average_bin = transformer([(average_bin[i] - average_bin[i-1]) for i in range(1, len(average_bin))])
diff_bin_0_24 = [bin_0[i]- bin_24[i] for i in range(len(bin_24))][1:]
diff_bin_0_0 = [(bin_0[i]- bin_0[i-1])/bin_0[i-1] for i in range(1, len(bin_0))]
# print(len(diff_average_bin), len(diff_bin_0_0), len(diff_bin_0_24))

data_diff = pd.DataFrame({'ddiff_average_bin': diff_average_bin, 'bin_0_0': diff_bin_0_0, 'bin_0_24': diff_bin_0_24})
# data_diff.corr()

In [31]:
def df2matrix(file_path:str, col_name: str=None, bin_num: bool= True):
    """
    转换数据结构，以date为横坐标，制定col_name为纵坐标
    """

    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df.set_index('date', inplace= True)
    df.sort_index(inplace= True)
    n, k = df.shape[0], 25
    # TODO: 1、把 bin0 分离出来，而后补充到特征里面去，也就是需要在  self.genNewFeatureBinVolume() 把bin0 特征融合进来
    if bin_num:
        data = pd.DataFrame(np.array(df[col_name]).reshape(int(n/k), k),
                                columns= ['bin{}'.format(i) for i in range(k)]).drop('bin0', axis=1)
    else:
        bin0_list = []
        for i in range(0, df.shape[0], 25):
            if i == 0:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i])/ df[col_name].iloc[i]])
            else:
                bin0_list.append([(df[col_name].iloc[i]- df[col_name].iloc[i-25])/ df[col_name].iloc[i-25]])
        result = np.array([[element[0]]*(k-1) for element in bin0_list])
        print(result.shape)
        data = pd.DataFrame(np.array(result),
                            columns= ['bin0' for i in range(k-1)])
    data['date'] = pd.to_datetime(list(sorted(set(df.index))), format='%Y/%m/%d')
    data.set_index('date', inplace= True)
    data.sort_index(inplace=True)
    return data

data = df2matrix(file_path= '../data/0308/000046_XSHE_25_daily.csv', col_name= 'bin_volume', bin_num= False)

(197, 24)


In [8]:
import logging
import requests

# 配置日志记录器
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler('api_log.log'),  # 日志文件处理器
                        logging.StreamHandler()  # 控制台日志处理器
                    ])

logger = logging.getLogger(__name__)

def analyze_sentiment(text, api_endpoint, api_key):
    headers = {"Authorization": f"Bearer {api_key}"}
    payload = {"text": text}
    
    try:
        response = requests.post(api_endpoint, headers=headers, json=payload)
        response.raise_for_status()  # 检查响应状态码
        
        # 假设API返回的JSON格式如下：{"sentiment": "positive", "confidence": 0.95}
        sentiment_result = response.json()
        
        # 记录情感分析结果
        logger.info(f"Sentiment analysis result for text '{text[:50]}' (truncated): {sentiment_result}")
        
        return sentiment_result
    except requests.exceptions.HTTPError as http_err:
        # 记录HTTP错误
        logger.error(f"HTTP error occurred: {http_err} - Status Code: {response.status_code}")
        logger.error(f"Response data: {response.text}")
    except requests.exceptions.RequestException as err:
        # 记录请求异常
        logger.error(f"Request exception occurred: {err}")
    except Exception as err:
        # 记录其他错误
        logger.error(f"An error occurred: {err}")

# 假设的API端点和API密钥
api_endpoint = "https://api.zhipu.ai/glm/sentiment"
api_key = "YOUR_API_KEY"
texts = ["这是一段正面文本", "这是一段负面文本"]

for text in texts:
    sentiment_result = analyze_sentiment(text, api_endpoint, api_key)
    # 这里可以添加更多的日志记录，例如记录分析结果


array([[2.1871026 , 2.03988665],
       [1.67665943, 1.5774206 ]])

In [112]:
import pandas as pd

df = pd.read_csv('../data/comment/000046.csv')
df['publishDate'] = pd.to_datetime(df['publishDate'])

start_time = '2020-5-12 15:00'
end_time = '2021-3-8 15:00'
# 2020-5-13 2021-3-8
filtered_df = df[(df['publishDate'] >= start_time) & (df['publishDate'] <= end_time)].copy()
filtered_df['dates'] = filtered_df['publishDate'].dt.date
filtered_df['times'] = filtered_df['publishDate'].dt.time
filtered_df.head(20)

,post_id,publishDate,newsTitle,posterId,posterName,dates,times
31858,929174946,2020-05-12 15:07:40,泛海下跌周期太长，被套资本太多，没有啥机会暴涨的，就像上次一样涨到8元就被打熄火,5323335728465078,股友oRx5gP,2020-05-12,15:07:40
31859,929207943,2020-05-12 17:26:28,民生证券将挥别北京 泛海控股再度布局上海滩,4506395531315256,投资者攻略,2020-05-12,17:26:28
31860,929210791,2020-05-12 17:39:29,适合在家创业的6个项目，有适合您的吗？,7301305858084640,小息谈资讯,2020-05-12,17:39:29
31861,929323847,2020-05-13 06:01:54,泛海，真的跌得好惨！,9273094111979056,水中月7878,2020-05-13,06:01:54
31862,929331305,2020-05-13 06:49:52,看看大盘走的多好，震荡上行，好多个股都跌惨了，什么破股市啊,3407145336565478,魅力的林顺昊,2020-05-13,06:49:52
31863,929345911,2020-05-13 08:06:38,泛海控股05月12日获深股通减仓27.45万股,8927005130571322,股友y31920576Y,2020-05-13,08:06:38
31864,929391203,2020-05-13 09:52:56,连烂海控股内部的员工都不买自己的股票，没有人比他们更懂得烂海[摊手],1481035284301320,ST烂海,2020-05-13,09:52:56
31865,929422014,2020-05-13 10:27:27,这货就没涨过吧,1077185584032898,股友EDCfg2,2020-05-13,10:27:27
31866,929431367,2020-05-13 10:40:53,这可怜的成交量,6303305832237758,纱布泥哟,2020-05-13,10:40:53
31867,929468369,2020-05-13 10:53:00,请问截至4月28日或者附近日子的股东人数多少？,6194084951661992,*ST泛海股友,2020-05-13,10:53:00


In [129]:
times_set = set()
for i in filtered_df['dates']:
    times_set.add(i)
data_bin = pd.DataFrame(index = list(times_set),
                        columns= [f'bin{i}' for i in range(0, 25)])
data_bin.index = pd.to_datetime(data_bin.index)
data_bin = data_bin.sort_index()
data_bin.head()
for day in data_bin.index[1:]:
    bin_newstitle = {}
    for i in range(0, 25):
        bin_newstitle[f'bin{i}'] = []
    bin_0 = [f'{day-pd.Timedelta(days=1)} 15:00', f'{day} 09:30']
    bin_13 = [f'{day} 11:30', f'{day} 13:00']
    # TODO: 直接书写不同的bin；或者直接循环函数输出不同的bin
    bin_newstitle['bin0'] = filtered_df[(filtered_df['publishDate'] >= bin_0[0]) & (filtered_df['publishDate'] <= bin_0[1])].loc[:, 'newsTitle'].values
    bin_newstitle['bin13'] = filtered_df[(filtered_df['publishDate'] >= bin_13[0]) & (filtered_df['publishDate'] <= bin_13[1])].loc[:, 'newsTitle'].values
    data_bin.loc[day] = bin_newstitle
    # print(day, )

In [134]:
data_bin.iloc[1, [0]].values

array([array(['泛海下跌周期太长，被套资本太多，没有啥机会暴涨的，就像上次一样涨到8元就被打熄火',
              '民生证券将挥别北京 泛海控股再度布局上海滩', '适合在家创业的6个项目，有适合您的吗？', '泛海，真的跌得好惨！',
              '看看大盘走的多好，震荡上行，好多个股都跌惨了，什么破股市啊', '泛海控股05月12日获深股通减仓27.45万股'],
             dtype=object)                                                 ],
      dtype=object)

In [ ]:
def calculate_time_bins(day):
    bins = []

    # bin_0: 从前一天的15:00到当天的09:30
    bin_0_start = f'{day - pd.Timedelta(days=1)} 15:00'
    bin_0_end = f'{day} 09:30'
    bins.append((bin_0_start, bin_0_end))

    # bin_1: 从bin_0的结束时间到结束时间加9.875分钟
    bin_1_start = bin_0_end
    bin_1_end = pd.to_datetime(bin_1_start) + pd.Timedelta(minutes=9.875)
    bins.append((bin_1_start, bin_1_end.strftime('%Y-%m-%d %H:%M')))

    # bin_2: 从bin_1的结束时间到结束时间加9.875分钟
    bin_2_start = bin_1_end.strftime('%Y-%m-%d %H:%M')
    bin_2_end = pd.to_datetime(bin_2_start) + pd.Timedelta(minutes=9.875)
    bins.append((bin_2_start, bin_2_end.strftime('%Y-%m-%d %H:%M')))

    # 可以继续添加更多的bin，如果需要

    return bins